<a href="https://colab.research.google.com/github/kingketan9/AdvancedDSA-Labs/blob/main/Ass5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Write a program for B tree having functions for the following operations:
  Insert an element (no duplicates are allowed),
  Delete an existing element,
  Traverse the B Tree (in-order, pre-order, and post-order)

INPUT:
Line 1 contains an integer NQ, the number of queries.
Line 2 contains the value for a minimum number of child pointers of a B tree node.

The next NQ lines contain queries and are of the form 'i xx' (Insert xx into a B tree) or 'd xx' (Delete xx from a B tree).

OUTPUT:
Output is a three-line answer printing number of split operations, the number of merge operations, and the tree traversal of a B tree that results after the execution of all NQ queries.

The first line is the total number of split operations performed.
The second line is the total number of merge operations performed.
Third line is the 'Inorder traversal'.

In [12]:
class BTreeNode:
    def __init__(self, keys=None, children=None):
        self.keys = []
        self.children = []
        
    def is_leaf(self):
        return not self.children
    
    def is_full(self, t):
        return len(self.keys) == 2*t - 1
    
    def search(self, k):
        i = 0
        while i < len(self.keys) and k > self.keys[i]:
            i += 1
        if i < len(self.keys) and k == self.keys[i]:
            return (self, i)
        elif self.is_leaf():
            return None
        else:
            return self.children[i].search(k)
        
    def split_child(self, i, t):
        z = BTreeNode()
        y = self.children[i]
        z.children = y.children[t:]
        y.children = y.children[:t]
        z.keys = y.keys[t:]
        y.keys = y.keys[:t-1]
        self.children.insert(i+1, z)
        self.keys.insert(i, y.keys[t-1])
        
    def insert(self, k, t):
        i = len(self.keys) - 1
        if self.is_leaf():
            self.keys.append(None)
            while i >= 0 and k < self.keys[i]:
                self.keys[i+1] = self.keys[i]
                i -= 1
            self.keys[i+1] = k
        else:
            while i >= 0 and k < self.keys[i]:
                i -= 1
            i += 1
            if self.children[i].is_full(t):
                self.split_child(i, t)
                if k > self.keys[i]:
                    i += 1
            self.children[i].insert(k, t)
            
    def minimum_key(self):
        if self.is_leaf():
            return self.keys[0]
        else:
            return self.children[0].minimum_key()
        
    def maximum_key(self):
        if self.is_leaf():
            return self.keys[-1]
        else:
            return self.children[-1].maximum_key()
        
    def predecessor(self, k):
        i = 0
        while i < len(self.keys) and k > self.keys[i]:
            i += 1
        if self.is_leaf():
            if i > 0:
                return self.keys[i-1]
            else:
                return None
        else:
            if i > 0:
                return self.children[i-1].maximum_key()
            else:
                return self.children[i].predecessor(k)
        
    def successor(self, k):
        i = 0
        while i < len(self.keys) and k >= self.keys[i]:
            i += 1
        if self.is_leaf():
            if i < len(self.keys):
                return self.keys[i]
            else:
                return None
        else:
            if i < len(self.children):
                return self.children[i].minimum_key()
            else:
                return self.children[i-1].successor(k)
    
    def delete(self, k, t):
        i = 0
        while i < len(self.keys) and k > self.keys[i]:
            i += 1
        if i < len(self.keys) and k == self.keys[i]:
            if self.is_leaf():
                del self.keys[i]
            else:
                if len(self.children[i]) >= t:
                    predecessor = self.children[i].maximum_key()
                    self.keys[i] = predecessor
                    self.children[i].delete(predecessor, t)
                elif len(self.children[i+1]) >= t:
                    successor = self.children[i+1].minimum_key()
                    self.keys[i] = successor
                    self.children[i+1].delete(successor, t)
                else:
                    self.merge_children(i, t)
                    self.children[i].delete(k, t)
        else:
            if self.is_leaf():
                return
            elif len(self.children[i]) < t:
                self.fix_child(i, t)
            self.children[i].delete(k, t)
                
    def merge_children(self, i, t):
        child1 = self.children[i]
        child2 = self.children[i+1]
        child1.keys.append(self.keys[i])
        child1.keys.extend(child2.keys)
        child1.children.extend(child2.children)
        del self.keys[i]
        del self.children[i+1]
        
    def fix_child(self, i, t):
        if i > 0 and len(self.children[i-1]) >= t:
            self.borrow_from_previous(i)
        elif i < len(self.children) - 1 and len(self.children[i+1]) >= t:
            self.borrow_from_next(i)
        else:
            if i < len(self.children):
                self.merge_children(i, t)
            else:
                self.merge_children(i-1, t)
                
    def borrow_from_previous(self, i):
        child1 = self.children[i]
        child2 = self.children[i-1]
        child1.children.insert(0, child2.children.pop())
        child1.keys.insert(0, self.keys.pop(i-1))
        self.keys.insert(i-1, child2.keys.pop())
        
    def borrow_from_next(self, i):
        child1 = self.children[i]
        child2 = self.children[i+1]
        child1.children.append(child2.children.pop(0))
        child1.keys.append(self.keys.pop(i))
        self.keys.insert(i, child2.keys.pop(0))

    def traverse_inorder(self):
        result = []
        n = len(self.keys)
        if self.is_leaf:
            for i in range(n):
                result.append(self.keys[i])
        else:
            for i in range(n):
                result.extend(self.child[i].traverse_inorder())
                result.append(self.keys[i])
            result.extend(self.child[n].traverse_inorder())
        return result

    def traverse_preorder(self):
        result = []
        n = len(self.keys)
        if not self.is_leaf:
            for i in range(n):
                result.append(self.keys[i])
                result.extend(self.child[i].traverse_preorder())
            result.extend(self.child[n].traverse_preorder())
        else:
            for i in range(n):
                result.append(self.keys[i])
        return result

    def traverse_postorder(self):
        result = []
        n = len(self.keys)
        if not self.is_leaf:
            for i in range(n):
                result.extend(self.child[i].traverse_postorder())
            result.extend(self.child[n].traverse_postorder())
            for i in range(n):
                result.append(self.keys[i])
        else:
            for i in range(n):
                result.append(self.keys[i])
        return result

tree = BTreeNode()
tree.insert(100,4)
tree.insert(50,4)
tree.insert(150,4)
tree.insert(25,4)
tree.insert(75,4)
tree.insert(125,4)
tree.traverse_inorder()

[25, 50, 75, 100, 125, 150]